<a href="https://colab.research.google.com/github/ahyz0569/Final-project-practice/blob/colab/200328_doc2vec_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doc2vec 사용하여 description 컬럼 분석하기

description 컬럼에 있는 문장들의 유사도를 확인해본다

colab 환경에서 필요한 라이브러리 설치

In [0]:
# mecab 설치
# 출처: https://somjang.tistory.com/entry/Google-Colab에서-Mecab-koMecab-ko-dic-쉽게-사용하기 [솜씨좋은장씨]

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-03-28 05:54:52--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=BEh17k16oHSK%2BN5CYgIRwtZQnoc%3D&Expires=1585376693&AWSAccessKeyId=AKIA6KOSE3

## Library import

In [0]:
import gensim
import sys
import pandas as pd
from konlpy.tag import Mecab
m = Mecab()
import re
import os
import numpy as np

# imports package
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer

## 데이터 로드 및 전처리

In [0]:
DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'

In [0]:
df = pd.read_csv('/content/drive/My Drive/Final-project/NLP/data/crawling_200326.csv')

In [0]:
df

,id,title,writer,viewers,size,time,level,ingredients,description,total_quantity,main,main_quantity,minor,minor_quantity,source,source_quantity,step0,img0,step1,img1,step2,img2,step3,img3,step4,img4,step5,img5,step6,img6,step7,img7,step8,img8,step9,img9,step10,img10,step11,img11,step12,img12,step13,img13,step14,img14,step15,img15,step16,img16,step17,img17,step18,img18,step19,img19,step20,img20,step21,img21,step22,img22,step23,img23,step24,img24,ingre_main_oneline,steps,steps_img,ingredients_tagged,main_tagged,minor_tagged
0,6924266,밥솥 오리고기 짜장라면(몸에 열을 내주는 겨울간식),밥솥개미,6498.0,1.0,30.0,아무나,"['오리고기 3조각, 짜장라면 1개, 물 300ml']",짜장라면에 오리고기가 잘 맞아요.,[''],"['오리고기 3조각, 짜장라면 1개, 물 300ml']",[''],[],[],[],[],1인분이라 3조각씩 따로 소분해서 얼려놓았던 오리고기를 해동시키고나서 물300ml와...,https://recipe1.ezmember.co.kr/cache/recipe/20...,고기를 완전히 익히기 위해 재가열 9분 끝난 후 다시 재가열 9분 해주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...,재가열이 다 된 후 그대로 짜장라면 면과 건더기 스프를 넣고 물에 고루 잠기게끔 면...,https://recipe1.ezmember.co.kr/cache/recipe/20...,재가열이 다 된 후 남은 물을 따라 버리고 짜장 스프 넣고 잘 비벼주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...,색감을 좋게 하려면 파를 얇게 다져서 위에 올려 드시면 보기에 좋아요,https://recipe1.ezmember.co.kr/cache/recipe/20...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,오리고기 짜장라면 물,['1인분이라 3조각씩 따로 소분해서 얼려놓았던 오리고기를 해동시키고나서 물300m...,['https://recipe1.ezmember.co.kr/cache/recipe/...,"['오리고기', '짜장라면']","['오리고기', '짜장라면']",[]
1,6912734,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,루던,47436.0,1.0,15.0,아무나,"['새송이버섯', '버터', '어린잎채소', '간장', '맛술', '올리고당']",지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가,"['2개', '30g', '', '1T', '1T', '1/2T']","['새송이버섯', '버터', '어린잎채소', '간장', '맛술', '올리고당']","['2개', '30g', '', '1T', '1T', '1/2T']",[],[],[],[],"버섯의 밑동과 갓을 잘라낸 뒤, 3~4등분 한다",https://recipe1.ezmember.co.kr/cache/recipe/20...,자른버섯의 양면에 칼집을 낸다,https://recipe1.ezmember.co.kr/cache/recipe/20...,팬에 버터를 녹인다,https://recipe1.ezmember.co.kr/cache/recipe/20...,버섯을 굽는다,https://recipe1.ezmember.co.kr/cache/recipe/20...,불을 약불로 줄이고 소스를 넣고 졸인다,https://recipe1.ezmember.co.kr/cache/recipe/20...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,새송이버섯 버터 어린잎채소 간장 맛술 올리고당,"['버섯의 밑동과 갓을 잘라낸 뒤, 3~4등분 한다', '자른버섯의 양면에 칼집을 ...",['https://recipe1.ezmember.co.kr/cache/recipe/...,"['송이버섯', '버터', '어린잎채소', '간장', '맛술', '올리고당']","['송이버섯', '버터', '어린잎채소', '간장', '맛술', '올리고당']",[]
2,6894323,고기처럼 쫄깃한 밥도둑 반찬 '새송이버섯 간장버터구이'레시피,뽀유TV,302564.0,2.0,10.0,아무나,"['새송이 버섯', '청양고추', '대파', '버터', '간장', '올리고당', '...",마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...,"['3동', '1개', '1/2대', '1T', '5T', '2T', '1T', '...","['새송이 버섯', '청양고추', '대파', '버터']","['3동', '1개', '1/2대', '1T']","['간장', '올리고당', '다진마늘', '참기름', '물']","['5T', '2T', '1T', '1T', '2T']",[],[],먼저 새송이 버섯 3동은 밑동을 제거하고 일정한 두께로 잘라 줍니다. 대파와 청양고...,https://recipe1.ezmember.co.kr/cache/recipe/20...,"간장5T,올리고당2T,물2T,다진마늘1T, 참기름1T를 섞어주신뒤 썰어놓은 청양고추...",https://recipe1.ezmember.co.kr/cache/recipe/20...,달궈진 팬에 버터 1스푼 녹여준 뒤 새송이버섯을 앞뒤로 구워 줍니다.중약불,https://recipe1.ezmember.co.kr/cache/recipe/20...,버섯이 쫄깃하게 구워지면 만들어 놓은 양념을 버섯이 자작히 잠기도록 부어 줍니다.약...,https://recipe1.ezmember.co.kr/cache/recipe/20...,고기처럼 쫄깃한 새송이버섯 간장버터구이 완성입니다!,https://recipe1.ezmember.co.kr/cache/recipe/20...,한개 먹어보니 고소한 버터향에 짭쪼롬한 간장양념이 어우려져 참 맛있어요!밥 한공기 ...,https://recipe1.ezmember.co.kr/cache/recipe/20...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,새송이 버섯 청양고추 대파 버터,['먼저 새송이 버섯 3동은 밑동을 제거하고 일정한 두께로 잘라 줍니다. 대파와 청...,['https://recipe1.ezmember.co.kr/cache/recipe/...,"['새송이', '버섯', '청양고추', '대파', '버터', '간장', '올리고당'...","['새송이', '버섯', '청양고추', '대파', '버터']","['간장', '올리고당', '마늘', '참기름']"
3,6899265,해물찜처럼 맛있는 백종원 소시지콩나물찜,뽀유TV,114302.0,3.0,30.0,아무나,"['콩나물', '비엔나소시지', '대파', '양파', '믈', '고추장', '고추가...",평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...,"['1봉', '1봉', '1대', '1/2개',

In [0]:
df.columns

Index(['id', 'title', 'writer', 'viewers', 'size', 'time', 'level',
       'ingredients', 'description', 'total_quantity', 'main', 'main_quantity',
       'minor', 'minor_quantity', 'source', 'source_quantity', 'step0', 'img0',
       'step1', 'img1', 'step2', 'img2', 'step3', 'img3', 'step4', 'img4',
       'step5', 'img5', 'step6', 'img6', 'step7', 'img7', 'step8', 'img8',
       'step9', 'img9', 'step10', 'img10', 'step11', 'img11', 'step12',
       'img12', 'step13', 'img13', 'step14', 'img14', 'step15', 'img15',
       'step16', 'img16', 'step17', 'img17', 'step18', 'img18', 'step19',
       'img19', 'step20', 'img20', 'step21', 'img21', 'step22', 'img22',
       'step23', 'img23', 'step24', 'img24', 'ingre_main_oneline', 'steps',
       'steps_img', 'ingredients_tagged', 'main_tagged', 'minor_tagged'],
      dtype='object')

In [0]:
df.shape

(95698, 72)

In [0]:
df['description'][0:10]

0                   짜장라면에 오리고기가 잘 맞아요.                
1       지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가                
2    마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...
3    평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...
4    손님상차림으로도 좋은 차돌박이찜 입니다 만들기도 너무쉽고 모양도 예뻐서 추천하는 메...
5    냉동실에 묵혀두었던 냉동새우 !  간단하게 맛있는 간식으로 만들어봤어요 ~ 술안주나...
6    봄부추는 인삼이나 녹용과도 바꾸지 않는다는 말이 있을 정도로 보약이라고 합니다. 부...
7    뜨끈한 밥에 대패삼겹살 한점! 집밥백선생 강추레시피! 간단하고 빠르게 만드는 별미메...
8      간단하지만 세상 시원한! 담백한 새우연두부달걀탕입니다:)                
9    언제나 사랑하는 치느님이지만 매번 시켜먹기도 부담스럽잖아요. 그렇다고 생닭을 가지고...
Name: description, dtype: object

In [0]:
## 특수문자 제거함수(기호들도 추가)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☆★♡♥]', '', readData)
    return text

In [0]:
# 명사와 형용사만 찾을 정규 표현식 생성
find_pos = re.compile(r'^N.*|VA|^M.*|XR(?!VV+)')

# 문장을 토큰화 한 후 명사와 형용사만 추가하는 함수 선언
def preprocessing_text(doc):
    doc = cleanText(str(doc))
    token_list=[]
    tokens = m.pos(doc, flatten=True)
    for word, pos in tokens:
        if find_pos.match(pos):
            token_list.append(word)
    return token_list

In [0]:
desc_df = df[['id', 'description']]
desc_df

,id,description
0,6924266,짜장라면에 오리고기가 잘 맞아요.
1,6912734,지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가
2,6894323,마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...
3,6899265,평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...
4,6881099,손님상차림으로도 좋은 차돌박이찜 입니다 만들기도 너무쉽고 모양도 예뻐서 추천하는 메...
...,...,...
95693,6844474,고소하고 부드러운 우유와 생크림에 쌉쌀한 초코와 상큼한 딸기가 환상의 디저트를 만들...
95694,6844473,야채가 더 많이 들어가서 위에 부담도 적고 오리훈제 야채샐러드 한접시면 양양가득 이...
95695,6844471,딸 친구가 갑자기 놀러 왔어요. 밥통에 밥은 있는데 반찬이 영~ 그래서 냉동실에 손...
95696,6844470,집에서 직접키운 바질로 만든 바질페스토! 시금치를 곁들여 만든 시금치 바질페스토 파...


In [0]:
desc_df['desc_token'] = desc_df['description'].apply(preprocessing_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
desc_df.head()

,id,description,desc_token
0,6924266,짜장라면에 오리고기가 잘 맞아요.,"[짜장, 라면, 오리고기, 잘]"
1,6912734,지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가,"[지금, 이런, 버섯, 없, 이것, 관자, 버섯]"
2,6894323,마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...,"[마땅, 반찬, 없, 때, 새, 송이버섯, 간장, 버터, 구, 이, 한, 번, 단,..."
3,6899265,평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...,"[평소, 매콤, 해물, 찜, 참, 좋, 해물, 찜, 맛있, 백, 주부, 소시지, 콩..."
4,6881099,손님상차림으로도 좋은 차돌박이찜 입니다 만들기도 너무쉽고 모양도 예뻐서 추천하는 메...,"[손님, 상차림, 좋, 차돌박이, 찜, 너무, 쉽, 모양, 예뻐서, 추천, 메뉴, ..."


## Doc2vec 모델 학습시키기 (1차)

In [0]:
# Doc2vec 입력 형식에 맞게 데이터 수정
# tags = 문서 ID
# words = 단어 list
desc_doc_df = desc_df[['id', 'desc_token']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in desc_doc_df]

In [0]:
# Training
max_epochs=10

model=Doc2Vec(window=10,
             vector_size=150,
             alpha=0.025,
             min_alpha=0.025,
             min_count=2,
             dm = 1,
             negative = 5,
             seed=9999)

model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
               total_examples=model.corpus_count,
               epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    model.min_alpha = model.alpha

# 모델 저장하기
from gensim.test.utils import get_tmpfile

# temp_path = get_tmpfile('title_doc2vec_0327.pkl')
DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
model.save(os.path.join(DOC2VEC_DRIVE_PATH, "desc_doc2vec_0328.pkl"))

model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# model load
DOC2VEC_MODEL_PATH = os.path.join(DOC2VEC_DRIVE_PATH, "desc_doc2vec_0328.pkl")
model = Doc2Vec.load(DOC2VEC_MODEL_PATH)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 새로운 문장을 넣어 유사도 확인하기
model.random.seed(8000)

example_list = preprocessing_text('부타동은 일본식 돼지고기 덮밥으로 \
부타는 일본어로 돼지를 뜻하고, 동은 밥을 뜻합니다.\
일본식 소스로 돼지고기를 조려서 밥위에 올린 덮밥식 음식인데요..\
부타동은 주로 목살 부위를 사용하지만 \
저는 대패삼겹살을 이용해서 만들었습니다. \
대패삼겹살은 양념 베이기가 더 쉽고, \
조리시간을 줄일 수 있어서 사용했구요.\
돼지고기는 다른 부위를 사용하셔도 상관없습니다.')
inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
    for des in desc_df[desc_df['id'] == rd[0]]['description']:
        print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6867842 0.8273130059242249 살림 9단의 만물상 186회 방송일 : 2017.04.02 동영상 출처 : 살림 9단의 만물상 - 곰TV                
6860087 0.8265597224235535 오늘 뭐 먹지? 195회 방송일 : 2016.10.13 동영상 출처 : 오늘 뭐 먹지? - 곰tv                
6858997 0.8248567581176758 오늘 뭐 먹지? 76회 방송일 : 2015.07.02 동영상 출처 : 오늘 뭐 먹지? - 곰tv                
6869469 0.8210789561271667 살림 9단의 만물상 190회 방송일 : 2017.04.30 동영상 출처 : 살림 9단의 만물상 - 곰TV                
6860674 0.8195765018463135 살림 9단의 만물상 128회 방송일 : 2016.2.21 동영상 출처 : 살림 9단의 만물상 - 곰TV                
6859633 0.8189240097999573 • 마이 리틀 텔레비전 73회  • 방송일 : 2016. 10.29                
6862001 0.8178478479385376 살림 9단의 만물상 166회 방송일 : 2016.11.13 동영상 출처 : 살림 9단의 만물상 - 곰TV                
6860691 0.813782811164856 살림 9단의 만물상 132회 방송일 : 2016.3.20 동영상 출처 : 살림 9단의 만물상 - 곰TV                
6860218 0.8106826543807983 • 마이 리틀 텔레비전 74회  • 방송일 : 2016. 11.05                
6886128 0.8104594945907593 알토란 129회 방송일 : 2017.06.04 동영상 출처 : 알토란 - 곰tv                
6910938 0.808917760848999 알토란 227회 방송일 : 2019.04

In [0]:
df[df['id']==6867842]

,id,title,writer,viewers,size,time,level,ingredients,description,total_quantity,main,main_quantity,minor,minor_quantity,source,source_quantity,step0,img0,step1,img1,step2,img2,step3,img3,step4,img4,step5,img5,step6,img6,step7,img7,step8,img8,step9,img9,step10,img10,step11,img11,step12,img12,step13,img13,step14,img14,step15,img15,step16,img16,step17,img17,step18,img18,step19,img19,step20,img20,step21,img21,step22,img22,step23,img23,step24,img24,ingre_main_oneline,steps,steps_img,ingredients_tagged,main_tagged,minor_tagged
81196,6867842,김현숙의 딸기 식초 - 살림 9단의 만물상 186회,살림9단의만물상,2417.0,6.0,60.0,초급,"['딸기', '설탕', '생막걸리', '소금', '생강편']",살림 9단의 만물상 186회 방송일 : 2017.04.02 동영상 출처 : 살림 9...,"['1kg', '300g', '450ml', '약간', '3쪽']","['딸기', '설탕', '생막걸리', '소금', '생강편']","['1kg', '300g', '450ml', '약간', '3쪽']",[],[],[],[],잘 씻어 물기를 뺀 딸기에 생 막걸리를 부어 조물조물 으깨준다.1. 종초를 사용할 ...,NaN,딸기를 으깬후 딸기 1kg에 설탕 300ml를 투하한다.,NaN,소금 1꼬집과 생강 2~3쪽을 넣어 섞어준다.,NaN,멸균한 병이나 항아리에 담아 깨끗한 면포로 덮어준다.,NaN,딸기 식초를 담근 후 2달간 숙성! 30도 미만의 반그늘에 2개월 정도 숙성기간이 ...,NaN,두 달 후에 체에 걸러 그늘에 보관하면 딸기 식초 완성!숙성 두 달 후면 바로 먹을...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기 설탕 생막걸리 소금 생강편,['잘 씻어 물기를 뺀 딸기에 생 막걸리를 부어 조물조물 으깨준다.1. 종초를 사용...,[],"['딸기', '설탕', '막걸리', '소금', '생강']","['딸기', '설탕', '막걸리', '소금', '생강']",[]


In [0]:
type(df['id'][0])

numpy.int64

In [0]:
df['id'][0]

6924266

## Description 전처리 2차

- description 데이터로 분석을 했더니 의미 없는 문장들이 검출됨
- 해서 의미 없는 description 데이터를 필터링 한 후 나온 데이터를 분석에 사용해야 할 것으로 보임

In [0]:
# 의미 없는 단어가 있는 행 거르는 함수
def find_filter_list(sentence):
    sentence = str(sentence)
    filter_list = ['방송일', '곰tv', '곰TV']
    for filter in filter_list:
        if filter in sentence:
            print(sentence)
            return 1
    
    return 0

In [0]:
desc_df['filter']=desc_df['description'].apply(find_filter_list)

· 집밥백선생3 16회 · 방송일 : 2017.05.30 · 동영상 출처 : 집밥백선생3 - 곰tv                
오늘 뭐 먹지? 110회 방송일 : 2015.11.19 동영상 출처 : 오늘 뭐 먹지? - 곰tv                
· 집밥백선생 21회 · 방송일 : 2015.10.06 · 동영상 출처 : 집밥백선생 - 곰TV                
· 집밥백선생3 12회 · 방송일 : 2017.05.02 · 동영상 출처 : 집밥백선생3 - 곰tv                
· 집밥백선생3 8회 · 방송일 : 2017.04.04 · 동영상 출처 : 집밥백선생3 - 곰tv                
· 집밥백선생3 15회 · 방송일 : 2017.05.23 · 동영상 출처 : 집밥백선생3 - 곰tv                
오늘 뭐 먹지? 125회 방송일 : 2016.01.11 동영상 출처 : 오늘 뭐 먹지? - 곰tv                
· 집밥백선생 19회 · 방송일 : 2015.09.22 · 동영상 출처 : 집밥백선생 - 곰TV                
· 집밥백선생3 5회 · 방송일 : 2017.03.14 · 동영상 출처 : 집밥백선생3 - 곰tv                
집밥백선생3 31회 방송일 : 2017.09.12 동영상 출처 : 집밥백선생3 - 곰TV                
· 집밥백선생2 14회 · 방송일 : 2016.06.21 · 동영상 출처 : 집밥백선생2 - 곰TV                
· 집밥백선생2 33회 · 방송일 : 2016.11.01 · 동영상 출처 : 집밥백선생2 - 곰tv                
· 집밥백선생2 6회 · 방송일 : 2016.04.26 · 동영상 출처 : 집밥백선생2 - 곰TV                
집밥백선생3 34회 방송일 : 2017.10.03 동영상 출처 : 집밥백선생3 - 곰TV                
· 집밥백선생3 9회 ·

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
desc_df.head()

,id,description,filter
0,6924266,짜장라면에 오리고기가 잘 맞아요.,0
1,6912734,지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가,0
2,6894323,마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...,0
3,6899265,평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...,0
4,6881099,손님상차림으로도 좋은 차돌박이찜 입니다 만들기도 너무쉽고 모양도 예뻐서 추천하는 메...,0


In [0]:
desc_df[desc_df['filter']==1]

,id,description,filter
114,6870622,· 집밥백선생3 16회 · 방송일 : 2017.05.30 · 동영상 출처 : 집밥백...,1
206,6859143,오늘 뭐 먹지? 110회 방송일 : 2015.11.19 동영상 출처 : 오늘 뭐 먹...,1
354,6839935,· 집밥백선생 21회 · 방송일 : 2015.10.06 · 동영상 출처 : 집밥백선...,1
401,6869643,· 집밥백선생3 12회 · 방송일 : 2017.05.02 · 동영상 출처 : 집밥백...,1
412,6868016,· 집밥백선생3 8회 · 방송일 : 2017.04.04 · 동영상 출처 : 집밥백선...,1
...,...,...,...
95335,6845047,• 마이 리틀 텔레비전 43회 • 방송일 : 2016. 3. 5 ...,1
95530,6844734,냉장고를 부탁해 70회 방송일 : 2016.3.14 동영상 출처 : 곰TV ...,1
95531,6844733,냉장고를 부탁해 70회 방송일 : 2016.3.14 동영상 출처 : 곰TV ...,1
95532,6844732,냉장고를 부탁해 70회 방송일 : 2016.3.14 동영상 출처 : 곰TV ...,1


In [0]:
# 의미 없는 행 삭제
desc_df.drop(desc_df[desc_df['filter'] == 1].index, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
desc_df.shape

(92330, 3)

In [0]:
stop_words = ['방송', '출처', 'tv', '동영상']

In [0]:
# 문장을 토큰화 한 후 의미 없는 단어는 제하고 명사와 형용사만 추가하는 함수 선언
def preprocessing_text_v2(doc):
    doc = cleanText(str(doc))
    token_list=[]
    tokens = m.pos(doc, flatten=True)
    for word, pos in tokens:
        if find_pos.match(pos):
            if word not in stop_words:
                token_list.append(word)
    return token_list

In [0]:
desc_df['desc_token'] = desc_df['description'].apply(preprocessing_text_v2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
desc_df.head()

,id,description,filter,desc_token
0,6924266,짜장라면에 오리고기가 잘 맞아요.,0,"[짜장, 라면, 오리고기, 잘]"
1,6912734,지금까지 이런 버섯은 없었다. 이것은 관자인가 버섯인가,0,"[지금, 이런, 버섯, 없, 이것, 관자, 버섯]"
2,6894323,마땅한 반찬 없을때 새송이버섯 간장버터구이 한번 만들어 보세요!단짠단짠 양념에 고기...,0,"[마땅, 반찬, 없, 때, 새, 송이버섯, 간장, 버터, 구, 이, 한, 번, 단,..."
3,6899265,평소 매콤한 해물찜을 참 좋아하는데요. 해물찜처럼 맛있는 백주부님 소시지콩나물찜을 ...,0,"[평소, 매콤, 해물, 찜, 참, 좋, 해물, 찜, 맛있, 백, 주부, 소시지, 콩..."
4,6881099,손님상차림으로도 좋은 차돌박이찜 입니다 만들기도 너무쉽고 모양도 예뻐서 추천하는 메...,0,"[손님, 상차림, 좋, 차돌박이, 찜, 너무, 쉽, 모양, 예뻐서, 추천, 메뉴, ..."


## Doc2vec 모델 학습시키기 (2차)

In [0]:
# Doc2vec 입력 형식에 맞게 데이터 수정
# tags = 문서 ID
# words = 단어 list
desc_doc_df = desc_df[['id', 'desc_token']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in desc_doc_df]

In [0]:
# Training

%%time
max_epochs=10

model=Doc2Vec(window=10,
             vector_size=150,
             alpha=0.025,
             min_alpha=0.025,
             min_count=2,
             dm = 1,
             negative = 5,
             seed=9999)

model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
               total_examples=model.corpus_count,
               epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    model.min_alpha = model.alpha

# 모델 저장하기
# from gensim.test.utils import get_tmpfile
# temp_path = get_tmpfile('title_doc2vec_0327.pkl')

DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
model.save(os.path.join(DOC2VEC_DRIVE_PATH, "desc_doc2vec_0328_ver2.pkl"))

model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 41min 35s, sys: 2min 57s, total: 44min 33s
Wall time: 40min 6s


In [0]:
# model load
DOC2VEC_MODEL_PATH = os.path.join(DOC2VEC_DRIVE_PATH, "desc_doc2vec_0328_ver2.pkl")
model = Doc2Vec.load(DOC2VEC_MODEL_PATH)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 새로운 문장을 넣어 유사도 확인하기
model.random.seed(8000)

example_list = preprocessing_text('부타동은 일본식 돼지고기 덮밥으로 \
부타는 일본어로 돼지를 뜻하고, 동은 밥을 뜻합니다.\
일본식 소스로 돼지고기를 조려서 밥위에 올린 덮밥식 음식인데요..\
부타동은 주로 목살 부위를 사용하지만 \
저는 대패삼겹살을 이용해서 만들었습니다. \
대패삼겹살은 양념 베이기가 더 쉽고, \
조리시간을 줄일 수 있어서 사용했구요.\
돼지고기는 다른 부위를 사용하셔도 상관없습니다.')
inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
    for des in desc_df[desc_df['id'] == rd[0]]['description']:
        print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6920996 0.696296215057373 머랭을 넣은 수플레 치즈케이크                
6864301 0.6934148669242859 밥숟가락 계량 입니다.                
6861742 0.6863479018211365 조리기능사 실기 연습과제^^                
6914444 0.6846836805343628 버섯넣은 콩나물국입니다 ~~#                
6924367 0.6835278272628784 2020년 새해에 먹은 떡국이랍니다.                
6840315 0.683314323425293 김장철에 무채를 말려두었다가 겨우내 무쳐 먹으면 맛나요                
6831479 0.6829073429107666 차줌마의 제육볶음 황금레시피를 공개합니다~                
6865554 0.680255651473999 밥숟가락 계량 입니다.                
3008505 0.6761611104011536 안동찜닭 맛있게 만들어봤어요~                
6915377 0.6753063201904297 집에서 레스토랑 분위기 내면서 칼질해요~~#                
6873134 0.6749553680419922 존맛+존맛=핵존맛!                
6903524 0.6747013330459595 국민반찬 콩자반 딱딱하지 않고 부드럽게 만들어 볼께요.                
6914988 0.6739864349365234 마리텔에서 백종원선생님께서 알려주신 콩국수 레시피로 초간단 콩국수를 만들어보았습니다.                
895698 0.6726219058036804 케이크의 기본 케이크 시트입니다.                
6864170 0.6725759506225586 밥숟가락 계량 입니다.                
6854708 0.6725354194641113 한식조리기능사 자격증 실기동영

In [0]:
ex_txt = '부타동은 일본식 돼지고기 덮밥으로 \
부타는 일본어로 돼지를 뜻하고, 동은 밥을 뜻합니다.\
일본식 소스로 돼지고기를 조려서 밥위에 올린 덮밥식 음식인데요..\
부타동은 주로 목살 부위를 사용하지만 \
저는 대패삼겹살을 이용해서 만들었습니다. \
대패삼겹살은 양념 베이기가 더 쉽고, \
조리시간을 줄일 수 있어서 사용했구요.\
돼지고기는 다른 부위를 사용하셔도 상관없습니다.'

In [0]:
for word, pos in m.pos(ex_txt):
# for word, pos in tokenizer_Mecab_pos('뽀얀 국물이 일품인 바지락 칼국수'):
    # print(word, pos)
    if find_pos.match(pos):
        print(word, pos)
    print("="*50)

부타동 NNP
일본식 NNG
돼지고기 NNG
덮밥 NNG
부 NNG
일본어 NNG
돼지 NNG
뜻 NNG
동 NNG
밥 NNG
뜻 NNG
일본식 NNG
소스 NNG
돼지고기 NNG
밥 NNG
위 NNG
덮밥 NNG
식 NNB
음식 NNG
부타동 NNP
주로 MAG
목살 NNG
부위 NNG
사용 NNG
저 NP
대패 NNG
삼겹살 NNG
이용 NNG
대패 NNG
삼겹살 NNG
양념 NNG
베이기 NNG
더 MAG
쉽 VA
조리 NNG
시간 NNG
수 NNB
사용 NNG
돼지고기 NNG
다른 MM
부위 NNG
사용 NNG
상관없 VA


2차 학습 결과: description 문장 분석은 생각보다 결과가 잘 나오지 않는다 이유가 무엇일까 고민이 필요한 부분인 것 같다